In [ ]:
sc.version

In [ ]:
from operator import add 

In [ ]:
import objects

In [ ]:
data = sc.textFile("./carat-context-factor-data/carat-context-factors-percom.csv"  )

In [ ]:
data_rdd = data.map( lambda x: x.replace("'" , '').split(";") ).map( lambda line: objects.Carat(line) )

In [ ]:
data_rdd.take(3)[0]

In [ ]:
def pearson( rdd , x  , y  ):
    
    rdd_x = rdd.map( lambda obj:  obj.data[x] )
    rdd_y = rdd.map( lambda obj: obj.data[y] )
    x_mean = rdd_x.reduce( add ) / rdd_x.count()
    y_mean = rdd_y.reduce(add) / rdd_x.count()
        
    sigma_x = rdd_x.map( lambda f :  (f - x_mean)**2  ).reduce(add)
    sigma_y = rdd_y.map( lambda f: (f- y_mean)**2 ).reduce( add )
    
    rdd_x = rdd_x.map( lambda f : f - x_mean )
    rdd_y = rdd_y.map( lambda f : f - y_mean )
    
    sg = sigma_x*sigma_y
    
    pearson = (rdd_x*rdd_y).reduce(add)/sg 
    
    return pearson
    

In [ ]:
%timeit pearson( data_rdd ,"energyRate", "cpuUsage" )

In [ ]:
%timeit pearson( data_rdd ,"energyRate", "screenBright" )

In [ ]:
%timeit pearson( data_rdd ,"energyRate", "wifiSpeed" )

In [ ]:
%timeit pearson( data_rdd ,"energyRate", "wifiStre" )

# Ex 2

In [100]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType, FloatType

In [124]:
caratSchema = StructType([
    StructField( "energyRate" , FloatType()),
    StructField("batteryHealth", StringType()),
    StructField("batteryTemperature", FloatType()),
    StructField("batteryVoltage", FloatType()),
    StructField("cpuUsage" , FloatType()) , 
    StructField("distanceTraveled", IntegerType()),
    StructField( "mobileDataActivity" , StringType()),
    StructField("mobileDataStatus", StringType()),
    StructField("mobileNetworkType", StringType()),
    StructField("networkType", StringType()),
    StructField("roamingEnabled", StringType()),
    StructField("screenBrightness", FloatType()),
    StructField("wifiLinkSpeed", StringType()),
    StructField("wifiSignalStrength", IntegerType()),
])

In [125]:
caratDf = spark.read.csv('./carat-context-factor-data/carat-context-factors-percom.csv', header=True, schema=caratSchema , sep = ";")

In [126]:
type( caratDf )

pyspark.sql.dataframe.DataFrame

In [127]:
caratDf.select("batteryTemperature").distinct().show()

+------------------+
|batteryTemperature|
+------------------+
|              18.0|
|              64.0|
|              47.0|
|               9.0|
|              58.0|
|             -11.0|
|             -40.0|
|              39.0|
|               5.0|
|              17.0|
|              30.0|
|       -4.350208E7|
|             -17.0|
|              90.0|
|              26.0|
|              41.0|
|             -16.0|
|              29.0|
|              -9.0|
|              23.0|
+------------------+
only showing top 20 rows



In [128]:
caratDf.select("batteryVoltage").distinct().show()

+--------------+
|batteryVoltage|
+--------------+
|         3.703|
|         4.224|
|         3.467|
|         3.442|
|        3599.0|
|         7.515|
|          3.32|
|      3883.281|
|        11.857|
|         6.705|
|      3724.053|
|      3809.769|
|        12.209|
|        11.913|
|      3643.755|
|         2.826|
|      4157.583|
|      4015.224|
|          8.26|
|      4013.706|
+--------------+
only showing top 20 rows



In [129]:
#outliers 

out_batteryTemperature = caratDf.filter( caratDf.batteryTemperature < 0.0 ).count() + caratDf.filter( caratDf.batteryTemperature >= 50.0  ).count() 
out_batteryVoltage =  caratDf.filter( caratDf.batteryVoltage < 2.0 ).count() + caratDf.filter( caratDf.batteryVoltage >= 4.5  ).count() 

In [130]:
out_batteryTemperature

9030

In [131]:
out_batteryVoltage

293296

In [132]:
carat = caratDf.select([ "energyRate" ,  "cpuUsage" , "screenBrightness" , "wifiSignalStrength" , "wifiLinkSpeed" ])

In [136]:
carat = carat.filter( carat.cpuUsage >= 0.0 ).filter( carat.cpuUsage <= 1.0 )
carat = carat.filter( carat.screenBrightness >= 0 ).filter( carat.screenBrightness <= 255 )
carat = carat.filter( carat.wifiSignalStrength >  -100 ).filter( carat.wifiSignalStrength < 0 )

In [134]:
carat

DataFrame[energyRate: float, cpuUsage: float, screenBrightness: float, wifiSignalStrength: int, wifiLinkSpeed: string]

In [138]:
carat.corr(  "energyRate" , "cpuUsage" )

0.013069832948308456